# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 6 2022 2:38PM,238909,10,SONSB0001813,"Nextsource Biotechnology, LLC",Released
1,Jun 6 2022 2:38PM,238909,10,SONSB0001814,"Nextsource Biotechnology, LLC",Released
2,Jun 6 2022 2:38PM,238909,10,SONSB0001815,"Nextsource Biotechnology, LLC",Released
3,Jun 6 2022 2:34PM,238901,10,0085740677,ISDIN Corporation,Released
4,Jun 6 2022 2:34PM,238901,10,0085740673,ISDIN Corporation,Released
5,Jun 6 2022 2:34PM,238901,10,0085740680,ISDIN Corporation,Released
6,Jun 6 2022 2:34PM,238901,10,0085740679,ISDIN Corporation,Released
7,Jun 6 2022 2:34PM,238901,10,0085740678,ISDIN Corporation,Released
8,Jun 6 2022 2:34PM,238901,10,0085740681,ISDIN Corporation,Released
9,Jun 6 2022 2:34PM,238901,10,0085740685,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,238903,Released,1
33,238904,Released,1
34,238907,Released,13
35,238908,Released,14
36,238909,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238903,NaN,NaN,1.0
238904,NaN,NaN,1.0
238907,NaN,NaN,13.0
238908,NaN,NaN,14.0
238909,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238820,5.0,7.0,1.0
238838,0.0,0.0,1.0
238844,0.0,0.0,3.0
238852,0.0,1.0,1.0
238857,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238820,5,7,1
238838,0,0,1
238844,0,0,3
238852,0,1,1
238857,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238820,5,7,1
1,238838,0,0,1
2,238844,0,0,3
3,238852,0,1,1
4,238857,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238820,5,7,1
1,238838,,,1
2,238844,,,3
3,238852,,1,1
4,238857,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 6 2022 2:38PM,238909,10,"Nextsource Biotechnology, LLC"
3,Jun 6 2022 2:34PM,238901,10,ISDIN Corporation
63,Jun 6 2022 2:31PM,238908,19,"AdvaGen Pharma, Ltd"
77,Jun 6 2022 2:28PM,238907,16,Sartorius Bioprocess Solutions
85,Jun 6 2022 2:28PM,238907,16,Sartorius Lab Products and Service
90,Jun 6 2022 2:11PM,238904,19,"AdvaGen Pharma, Ltd"
91,Jun 6 2022 2:07PM,238903,21,"NBTY Global, Inc."
92,Jun 6 2022 2:06PM,238902,21,"NBTY Global, Inc."
93,Jun 6 2022 1:58PM,238899,10,Eywa Pharma Inc.
94,Jun 6 2022 1:57PM,238900,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 6 2022 2:38PM,238909,10,"Nextsource Biotechnology, LLC",,,3
1,Jun 6 2022 2:34PM,238901,10,ISDIN Corporation,,,60
2,Jun 6 2022 2:31PM,238908,19,"AdvaGen Pharma, Ltd",,,14
3,Jun 6 2022 2:28PM,238907,16,Sartorius Bioprocess Solutions,,,13
4,Jun 6 2022 2:28PM,238907,16,Sartorius Lab Products and Service,,,13
5,Jun 6 2022 2:11PM,238904,19,"AdvaGen Pharma, Ltd",,,1
6,Jun 6 2022 2:07PM,238903,21,"NBTY Global, Inc.",,,1
7,Jun 6 2022 2:06PM,238902,21,"NBTY Global, Inc.",,,1
8,Jun 6 2022 1:58PM,238899,10,Eywa Pharma Inc.,,,1
9,Jun 6 2022 1:57PM,238900,10,"Methapharm, Inc.",,,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 2:38PM,238909,10,"Nextsource Biotechnology, LLC",3,,
1,Jun 6 2022 2:34PM,238901,10,ISDIN Corporation,60,,
2,Jun 6 2022 2:31PM,238908,19,"AdvaGen Pharma, Ltd",14,,
3,Jun 6 2022 2:28PM,238907,16,Sartorius Bioprocess Solutions,13,,
4,Jun 6 2022 2:28PM,238907,16,Sartorius Lab Products and Service,13,,
5,Jun 6 2022 2:11PM,238904,19,"AdvaGen Pharma, Ltd",1,,
6,Jun 6 2022 2:07PM,238903,21,"NBTY Global, Inc.",1,,
7,Jun 6 2022 2:06PM,238902,21,"NBTY Global, Inc.",1,,
8,Jun 6 2022 1:58PM,238899,10,Eywa Pharma Inc.,1,,
9,Jun 6 2022 1:57PM,238900,10,"Methapharm, Inc.",10,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 2:38PM,238909,10,"Nextsource Biotechnology, LLC",3,,
1,Jun 6 2022 2:34PM,238901,10,ISDIN Corporation,60,,
2,Jun 6 2022 2:31PM,238908,19,"AdvaGen Pharma, Ltd",14,,
3,Jun 6 2022 2:28PM,238907,16,Sartorius Bioprocess Solutions,13,,
4,Jun 6 2022 2:28PM,238907,16,Sartorius Lab Products and Service,13,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 2:38PM,238909,10,"Nextsource Biotechnology, LLC",3.0,NaN,NaN
1,Jun 6 2022 2:34PM,238901,10,ISDIN Corporation,60.0,NaN,NaN
2,Jun 6 2022 2:31PM,238908,19,"AdvaGen Pharma, Ltd",14.0,NaN,NaN
3,Jun 6 2022 2:28PM,238907,16,Sartorius Bioprocess Solutions,13.0,NaN,NaN
4,Jun 6 2022 2:28PM,238907,16,Sartorius Lab Products and Service,13.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 2:38PM,238909,10,"Nextsource Biotechnology, LLC",3.0,0.0,0.0
1,Jun 6 2022 2:34PM,238901,10,ISDIN Corporation,60.0,0.0,0.0
2,Jun 6 2022 2:31PM,238908,19,"AdvaGen Pharma, Ltd",14.0,0.0,0.0
3,Jun 6 2022 2:28PM,238907,16,Sartorius Bioprocess Solutions,13.0,0.0,0.0
4,Jun 6 2022 2:28PM,238907,16,Sartorius Lab Products and Service,13.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2866659,110.0,19.0,0.0
12,238886,1.0,0.0,0.0
15,238852,1.0,1.0,0.0
16,477814,26.0,0.0,0.0
17,238873,1.0,0.0,0.0
18,477771,2.0,0.0,0.0
19,1194390,22.0,7.0,5.0
20,1194337,96.0,33.0,0.0
21,716699,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2866659,110.0,19.0,0.0
1,12,238886,1.0,0.0,0.0
2,15,238852,1.0,1.0,0.0
3,16,477814,26.0,0.0,0.0
4,17,238873,1.0,0.0,0.0
5,18,477771,2.0,0.0,0.0
6,19,1194390,22.0,7.0,5.0
7,20,1194337,96.0,33.0,0.0
8,21,716699,3.0,0.0,0.0
9,22,238869,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,110.0,19.0,0.0
1,12,1.0,0.0,0.0
2,15,1.0,1.0,0.0
3,16,26.0,0.0,0.0
4,17,1.0,0.0,0.0
5,18,2.0,0.0,0.0
6,19,22.0,7.0,5.0
7,20,96.0,33.0,0.0
8,21,3.0,0.0,0.0
9,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,110.0
1,12,Released,1.0
2,15,Released,1.0
3,16,Released,26.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,19,20,21,22
Status,,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
Executing,19.0,0.0,1.0,0.0,0.0,0.0,7.0,33.0,0.0,0.0
Released,110.0,1.0,1.0,26.0,1.0,2.0,22.0,96.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
1,Executing,19.0,0.0,1.0,0.0,0.0,0.0,7.0,33.0,0.0,0.0
2,Released,110.0,1.0,1.0,26.0,1.0,2.0,22.0,96.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
1,Executing,19.0,0.0,1.0,0.0,0.0,0.0,7.0,33.0,0.0,0.0
2,Released,110.0,1.0,1.0,26.0,1.0,2.0,22.0,96.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()